In [1]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\general_functions\Extracting Data Functions.ipynb"


In [2]:
f_name = os.path.join(os.path.dirname(os.getcwd()),"Nespresso Australia Dataset.xlsx")

ManufOrTopC ="Top Companies" #Manufacturer or Top Companies
BrandOrTopB= "Top Brands" #Brand or Top Brands
prodORitem="Product"

client_manuf = ["Nespresso"]
client_brands = ["Nespresso Original Capsules", "Nespresso Vertuo Capsules"]

decimals = 2
sign = "before"
currency = '$'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

categories = ["Coffee Capsules"]
sectors = ["Nespresso Original Compatible","Nespresso Vertuo Compatible"]
segments = ["Caffeinated","Decaf","Flavored"]
subsegments= []
subcategories= []

national = False
customareas= ""
areas = ["CHANNEL"]
 
regions_RET  = []
channels_RET = []
market_RET = []
 
regions_CHAN = []
channels_CHAN = ["Grocery","Dtc"]
market_CHAN = []
 
regions_CUST = []
channels_CUST = []
market_CUST = []

data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending Dec 2024"
years = ['2022', '2023','2024']
end_date = "2025-01-01"

direct_parent = {"Sector":list(zip(["Category"]*len(categories),categories))
                ,"Segment":list(zip(["Sector"]*len(sectors),sectors))
                ,"SubSegment":list(zip(["Segment"]*len(segments),segments))
                ,"SubCategory":list(zip(["Segment"]*len(segments),segments))
                }




In [3]:
# Generate the past 12 months
past_12_months = pd.date_range(end=end_date , periods=12, freq='M').strftime('%b-%y').tolist()

regions = regions_RET + regions_CHAN + regions_CUST
channels = channels_RET + channels_CHAN + channels_CUST
markets = market_RET + market_CHAN + market_CUST

In [4]:
f_path = Path.cwd()

excel = client.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True  # False
wb = excel.Workbooks.Open(f_name)
ws=wb.Sheets([s.Name for s in wb.Sheets][0])
s_name = [s.Name for s in wb.Sheets][0]
## If changed we'll need to change the iloc's of the cleaning
pvtTable = ws.PivotTables(1)

#change report layout
pvtTable.RowAxisLayout(1)   #RowAxisLayout(1) for tabular form

#change pivot table style
#Select from Design tab, try out Medium9 or Medium3
pvtTable.TableStyle2 = "pivotStyleMedium21"
pvtTable.ClearTable()

pvtTable.TableRange2.Cut(ws.Range("A16"))


fieldsNamePosition={}
for i in range(1,pvtTable.CubeFields.Count+1):
    fieldsNamePosition[str(pvtTable.CubeFields(i))]=i

In [5]:
# filter_dictionary_keys(fieldsNamePosition, 'Top Brand')

start_time = time.time()

#### By Brand only SKU share

In [6]:

assortmentbrand={}

row_list=[f'[Products].[{BrandOrTopB}]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=[]
pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
if len(categories) != 0:
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 

value_list=['[Measures].[Value Share]','[Measures].[SKU Share]']
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

In [7]:

if len(categories) != 0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        wb.Save()
        con = pd.read_excel(f_name, sheet_name=s_name)        
        assortmentbrand[f'{categories[0]} | National'] = con
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()

    if len(regions) != 0:
        assortmentbrand = create_dataframe(regions,assortmentbrand,"[Market].[Region].[Region]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()

    if len(channels) != 0:
        assortmentbrand = create_dataframe(channels,assortmentbrand,"[Market].[Channel].[Channel]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()


    if len(markets) != 0:
        assortmentbrand = create_dataframe(markets,assortmentbrand,"[Market].[Market].[Market]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()


# Dataframes by months by Sectors for Areas = NATIONAL, RETAILER, CHANNEL,POS

if len(sectors)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'


    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortmentbrand = create_dataframe(sectors,assortmentbrand,'[Products].[Sector].[Sector]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

    if len(regions) != 0:
        assortmentbrand = create_dataframe(regions,assortmentbrand,"[Market].[Region].[Region]",pvtTable, '',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

    if len(channels) != 0:
        assortmentbrand = create_dataframe(channels,assortmentbrand,"[Market].[Channel].[Channel]",pvtTable,'',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()


    if len(markets) != 0:
        assortmentbrand = create_dataframe(markets,assortmentbrand,"[Market].[Market].[Market]",pvtTable,'',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()
        

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'

    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortmentbrand = create_dataframe(segments,assortmentbrand,'[Products].[Segment].[Segment]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

    if len(regions) != 0:
        assortmentbrand = create_dataframe(regions,assortmentbrand,"[Market].[Region].[Region]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

    if len(channels) != 0:
        assortmentbrand = create_dataframe(channels,assortmentbrand,"[Market].[Channel].[Channel]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()


    if len(markets) != 0:
        assortmentbrand = create_dataframe(markets,assortmentbrand,"[Market].[Market].[Market]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'

    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortmentbrand = create_dataframe(subsegments,assortmentbrand,'[Products].[SubSegment].[SubSegment]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

    if len(regions) != 0:
        assortmentbrand = create_dataframe(regions,assortmentbrand,"[Market].[Region].[Region]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

    if len(channels) != 0:
        assortmentbrand = create_dataframe(channels,assortmentbrand,"[Market].[Channel].[Channel]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()


    if len(markets) != 0:
        assortmentbrand = create_dataframe(markets,assortmentbrand,"[Market].[Market].[Market]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'

    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortmentbrand = create_dataframe(subcategories,assortmentbrand,'[Products].[SubCategory].[SubCategory]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    if len(regions) != 0:
        assortmentbrand = create_dataframe(regions,assortmentbrand,"[Market].[Region].[Region]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    if len(channels) != 0:
        assortmentbrand = create_dataframe(channels,assortmentbrand,"[Market].[Channel].[Channel]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()


    if len(markets) != 0:
        assortmentbrand = create_dataframe(markets,assortmentbrand,"[Market].[Market].[Market]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()



In [8]:
with open('Assortment Datasets/assortmentbybrand.pickle', 'wb') as handle:
    pickle.dump(assortmentbrand, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### By Manuf only SKU share

In [9]:
assortmentmanuf={}

row_list=[f'[Products].[{ManufOrTopC}]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=[]
pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
if len(categories) != 0:
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 

value_list=['[Measures].[Value Share]','[Measures].[SKU Share]']
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

In [10]:
if len(categories) != 0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        wb.Save()
        con = pd.read_excel(f_name, sheet_name=s_name)        
        assortmentmanuf[f'{categories[0]} | National'] = con
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()

    if len(regions) != 0:
        assortmentmanuf = create_dataframe(regions,assortmentmanuf,"[Market].[Region].[Region]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()

    if len(channels) != 0:
        assortmentmanuf = create_dataframe(channels,assortmentmanuf,"[Market].[Channel].[Channel]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()


    if len(markets) != 0:
        assortmentmanuf = create_dataframe(markets,assortmentmanuf,"[Market].[Market].[Market]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()


# Dataframes by months by Sectors for Areas = NATIONAL, RETAILER, CHANNEL,POS

if len(sectors)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'


    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortmentmanuf = create_dataframe(sectors,assortmentmanuf,'[Products].[Sector].[Sector]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

    if len(regions) != 0:
        assortmentmanuf = create_dataframe(regions,assortmentmanuf,"[Market].[Region].[Region]",pvtTable, '',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

    if len(channels) != 0:
        assortmentmanuf = create_dataframe(channels,assortmentmanuf,"[Market].[Channel].[Channel]",pvtTable,'',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()


    if len(markets) != 0:
        assortmentmanuf = create_dataframe(markets,assortmentmanuf,"[Market].[Market].[Market]",pvtTable,'',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()
        

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'

    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortmentmanuf = create_dataframe(segments,assortmentmanuf,'[Products].[Segment].[Segment]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

    if len(regions) != 0:
        assortmentmanuf = create_dataframe(regions,assortmentmanuf,"[Market].[Region].[Region]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

    if len(channels) != 0:
        assortmentmanuf = create_dataframe(channels,assortmentmanuf,"[Market].[Channel].[Channel]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()


    if len(markets) != 0:
        assortmentmanuf = create_dataframe(markets,assortmentmanuf,"[Market].[Market].[Market]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'

    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortmentmanuf = create_dataframe(subsegments,assortmentmanuf,'[Products].[SubSegment].[SubSegment]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

    if len(regions) != 0:
        assortmentmanuf = create_dataframe(regions,assortmentmanuf,"[Market].[Region].[Region]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

    if len(channels) != 0:
        assortmentmanuf = create_dataframe(channels,assortmentmanuf,"[Market].[Channel].[Channel]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()


    if len(markets) != 0:
        assortmentmanuf = create_dataframe(markets,assortmentmanuf,"[Market].[Market].[Market]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'

    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortmentmanuf = create_dataframe(subcategories,assortmentmanuf,'[Products].[SubCategory].[SubCategory]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    if len(regions) != 0:
        assortmentmanuf = create_dataframe(regions,assortmentmanuf,"[Market].[Region].[Region]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    if len(channels) != 0:
        assortmentmanuf = create_dataframe(channels,assortmentmanuf,"[Market].[Channel].[Channel]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()


    if len(markets) != 0:
        assortmentmanuf= create_dataframe(markets,assortmentmanuf,"[Market].[Market].[Market]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()


In [11]:
with open('Assortment Datasets/assortmentbymanuf.pickle', 'wb') as handle:
    pickle.dump(assortmentmanuf, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### By Product all


In [12]:
assortment = {}
assortment_manuf={}

In [13]:
# by product
pvtTable.ClearTable()
row_list=[]
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Value Share]','[Measures].[WD]','[Measures].[Cumulative ' + prodORitem + ' Share]','[Measures].[' + prodORitem + ' Sales Rate]', '[Measures].[Net Sales]', '[Measures].[Trade Margin %]']

pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
if len(categories) != 0:
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 

row_list=[f"[Products].[{BrandOrTopB}]",f'[Products].[{prodORitem}]']
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


In [14]:
if len(categories) != 0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        wb.Save()
        con = pd.read_excel(f_name, sheet_name=s_name)        
        assortment[f'{categories[0]} | National'] = con
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()

    if len(regions) != 0:
        assortment = create_dataframe(regions,assortment,"[Market].[Region].[Region]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()

    if len(channels) != 0:
        assortment = create_dataframe(channels,assortment,"[Market].[Channel].[Channel]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()


    if len(markets) != 0:
        assortment = create_dataframe(markets,assortment,"[Market].[Market].[Market]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()


# Dataframes by months by Sectors for Areas = NATIONAL, RETAILER, CHANNEL,POS

if len(sectors) != 0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'

    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortment = create_dataframe(sectors,assortment,'[Products].[Sector].[Sector]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()
        

    if len(regions) != 0:
        assortment = create_dataframe(regions,assortment,"[Market].[Region].[Region]",pvtTable, '',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

    if len(channels) != 0:
        assortment = create_dataframe(channels,assortment,"[Market].[Channel].[Channel]",pvtTable,'',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()


    if len(markets) != 0:
        assortment = create_dataframe(markets,assortment,"[Market].[Market].[Market]",pvtTable,'',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()
        
        


if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortment = create_dataframe(segments,assortment,'[Products].[Segment].[Segment]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

    if len(regions) != 0:
        assortment = create_dataframe(regions,assortment,"[Market].[Region].[Region]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

    if len(channels) != 0:
        assortment = create_dataframe(channels,assortment,"[Market].[Channel].[Channel]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()


    if len(markets) != 0:
        assortment = create_dataframe(markets,assortment,"[Market].[Market].[Market]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()
    
if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortment = create_dataframe(subsegments,assortment,'[Products].[SubSegment].[SubSegment]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

    if len(regions) != 0:
        assortment = create_dataframe(regions,assortment,"[Market].[Region].[Region]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

    if len(channels) != 0:
        assortment = create_dataframe(channels,assortment,"[Market].[Channel].[Channel]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()


    if len(markets) != 0:
        assortment = create_dataframe(markets,assortment,"[Market].[Market].[Market]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortment = create_dataframe(subcategories,assortment,'[Products].[SubCategory].[SubCategory]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    if len(regions) != 0:
        assortment = create_dataframe(regions,assortment,"[Market].[Region].[Region]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    if len(channels) != 0:
        assortment = create_dataframe(channels,assortment,"[Market].[Channel].[Channel]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()


    if len(markets) != 0:
        assortment = create_dataframe(markets,assortment,"[Market].[Market].[Market]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()


In [15]:
with open('Assortment Datasets/assortment.pickle', 'wb') as handle:
    pickle.dump(assortment, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
# by product
pvtTable.ClearTable()
row_list=[]
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Value Share]','[Measures].[WD]','[Measures].[Cumulative ' + prodORitem + ' Share]','[Measures].[' + prodORitem + ' Sales Rate]', '[Measures].[Net Sales]', '[Measures].[Trade Margin %]']

pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
if len(categories) != 0:
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 

row_list=[f'[Products].[{ManufOrTopC}]',f'[Products].[{prodORitem}]']
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

In [17]:
if len(categories) != 0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        wb.Save()
        con = pd.read_excel(f_name, sheet_name=s_name)        
        assortment_manuf[f'{categories[0]} | National'] = con
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()

    if len(regions) != 0:
        assortment_manuf = create_dataframe(regions,assortment_manuf,"[Market].[Region].[Region]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()

    if len(channels) != 0:
        assortment_manuf = create_dataframe(channels,assortment_manuf,"[Market].[Channel].[Channel]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()


    if len(markets) != 0:
        assortment_manuf = create_dataframe(markets,assortment_manuf,"[Market].[Market].[Market]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()


# Dataframes by months by Sectors for Areas = NATIONAL, RETAILER, CHANNEL,POS

if len(sectors) != 0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'

    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortment_manuf = create_dataframe(sectors,assortment_manuf,'[Products].[Sector].[Sector]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()
        

    if len(regions) != 0:
        assortment_manuf = create_dataframe(regions,assortment_manuf,"[Market].[Region].[Region]",pvtTable, '',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

    if len(channels) != 0:
        assortment_manuf = create_dataframe(channels,assortment_manuf,"[Market].[Channel].[Channel]",pvtTable,'',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()


    if len(markets) != 0:
        assortment_manuf = create_dataframe(markets,assortment_manuf,"[Market].[Market].[Market]",pvtTable,'',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()
        
        


if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortment_manuf = create_dataframe(segments,assortment_manuf,'[Products].[Segment].[Segment]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

    if len(regions) != 0:
        assortment_manuf = create_dataframe(regions,assortment_manuf,"[Market].[Region].[Region]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

    if len(channels) != 0:
        assortment_manuf = create_dataframe(channels,assortment_manuf,"[Market].[Channel].[Channel]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()


    if len(markets) != 0:
        assortment_manuf = create_dataframe(markets,assortment_manuf,"[Market].[Market].[Market]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()
    
if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortment_manuf = create_dataframe(subsegments,assortment_manuf,'[Products].[SubSegment].[SubSegment]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

    if len(regions) != 0:
        assortment_manuf = create_dataframe(regions,assortment_manuf,"[Market].[Region].[Region]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

    if len(channels) != 0:
        assortment_manuf = create_dataframe(channels,assortment_manuf,"[Market].[Channel].[Channel]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()


    if len(markets) != 0:
        assortment_manuf = create_dataframe(markets,assortment_manuf,"[Market].[Market].[Market]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        assortment_manuf = create_dataframe(subcategories,assortment_manuf,'[Products].[SubCategory].[SubCategory]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    if len(regions) != 0:
        assortment_manuf = create_dataframe(regions,assortment_manuf,"[Market].[Region].[Region]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    if len(channels) != 0:
        assortment_manuf = create_dataframe(channels,assortment_manuf,"[Market].[Channel].[Channel]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()


    if len(markets) != 0:
        assortment_manuf = create_dataframe(markets,assortment_manuf,"[Market].[Market].[Market]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

In [18]:
with open('Assortment Datasets/assortment_manuf.pickle', 'wb') as handle:
    pickle.dump(assortment_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### By Product only for Cumulitive Share

In [19]:
cumulative_share = {}

In [20]:
row_list=[]
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Cumulative ' + prodORitem + ' Share]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

row_list=[f'[Products].[{prodORitem}]']
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

In [21]:
if len(categories) != 0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        wb.Save()
        con = pd.read_excel(f_name, sheet_name=s_name)        
        cumulative_share[f'{categories[0]} | National'] = con
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()

    if len(regions) != 0:
        cumulative_share = create_dataframe(regions,cumulative_share,"[Market].[Region].[Region]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()

    if len(channels) != 0:
        cumulative_share = create_dataframe(channels,cumulative_share,"[Market].[Channel].[Channel]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()

    if len(markets) != 0:
        cumulative_share = create_dataframe(markets,cumulative_share,"[Market].[Market].[Market]",pvtTable, f'{categories[0]} | ')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()


# Dataframes by months by Sectors for Areas = NATIONAL, RETAILER, CHANNEL,POS

if len(sectors) != 0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'

    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'


        cumulative_share = create_dataframe(sectors,cumulative_share,'[Products].[Sector].[Sector]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()



    if len(regions) != 0:
        cumulative_share = create_dataframe(regions,cumulative_share,"[Market].[Region].[Region]",pvtTable, '',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

    if len(channels) != 0:
        cumulative_share = create_dataframe(channels,cumulative_share,"[Market].[Channel].[Channel]",pvtTable,'',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()


    if len(markets) != 0:
        cumulative_share = create_dataframe(markets,cumulative_share,"[Market].[Market].[Market]",pvtTable,'',sectors,'[Products].[Sector].[Sector]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()
    


if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    if national:

        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        
        cumulative_share = create_dataframe(segments,cumulative_share,'[Products].[Segment].[Segment]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()

    if len(regions) != 0:
        cumulative_share = create_dataframe(regions,cumulative_share,"[Market].[Region].[Region]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

    if len(channels) != 0:
        cumulative_share = create_dataframe(channels,cumulative_share,"[Market].[Channel].[Channel]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()


    if len(markets) != 0:
        cumulative_share = create_dataframe(markets,cumulative_share,"[Market].[Market].[Market]",pvtTable,'',segments,'[Products].[Segment].[Segment]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()


if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'

    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        
        cumulative_share = create_dataframe(subsegments,cumulative_share,'[Products].[SubSegment].[SubSegment]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()

    if len(regions) != 0:
        cumulative_share = create_dataframe(regions,cumulative_share,"[Market].[Region].[Region]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

    if len(channels) != 0:
        cumulative_share = create_dataframe(channels,cumulative_share,"[Market].[Channel].[Channel]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()


    if len(markets) != 0:
        cumulative_share = create_dataframe(markets,cumulative_share,"[Market].[Market].[Market]",pvtTable,'',subsegments,'[Products].[SubSegment].[SubSegment]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'

    if national:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        
        cumulative_share = create_dataframe(subcategories,cumulative_share,'[Products].[SubCategory].[SubCategory]',pvtTable, 'National | ')
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()

    if len(regions) != 0:
        cumulative_share = create_dataframe(regions,cumulative_share,"[Market].[Region].[Region]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    if len(channels) != 0:
        cumulative_share = create_dataframe(channels,cumulative_share,"[Market].[Channel].[Channel]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()


    if len(markets) != 0:
        cumulative_share = create_dataframe(markets,cumulative_share,"[Market].[Market].[Market]",pvtTable,'',subcategories,'[Products].[SubCategory].[SubCategory]')
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

In [22]:
with open('Assortment Datasets/cumulative_share.pickle', 'wb') as handle:
    pickle.dump(cumulative_share, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Sectors Fair Share slide

In [23]:
for ke, val in direct_parent.items():
    print(ke)
    globals()[f"{ke}_dfs_total"] = {}
    globals()[f"{ke}_dfs_brands"] = {}
    globals()[f"{ke}_dfs_manuf"] = {}

    row_list=[ f'[Products].[{ke}]']
    column_list=[]
    filter_list=[f'[Products].[{ManufOrTopC}]','[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f"[Products].[{BrandOrTopB}]"]
    value_list=['[Measures].[Av No SKUs]', '[Measures].[SKU Share]','[Measures].[Value Share]']


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    if len(categories) != 0:
        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'


    # pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    # pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(globals()[f"{ke}_dfs_total"],'')
    
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
    calculate_data(globals()[f"{ke}_dfs_manuf"],f'{client_manuf[0]} | ')


    for brand in client_brands:
        pvtTable.PivotFields(f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]").ClearAllFilters()
        pvtTable.PivotFields(f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]").CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'
        calculate_data(globals()[f"{ke}_dfs_brands"],f'{brand} | ')



    with open(f'Assortment Datasets/{ke}_dfs_total.pickle', 'wb') as handle:
        pickle.dump(globals()[f"{ke}_dfs_total"], handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(f'Assortment Datasets/{ke}_dfs_brands.pickle', 'wb') as handle:
        pickle.dump(globals()[f"{ke}_dfs_brands"], handle, protocol=pickle.HIGHEST_PROTOCOL)    

    with open(f'Assortment Datasets/{ke}_dfs_manuf.pickle', 'wb') as handle:
        pickle.dump(globals()[f"{ke}_dfs_manuf"], handle, protocol=pickle.HIGHEST_PROTOCOL)    



Sector
Segment
SubSegment
SubCategory


In [24]:
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")


Script started at: Wed Mar 26 17:40:45 2025
Script ended at: Wed Mar 26 18:02:21 2025
Elapsed time: 1296.53 seconds
